In [ ]:
# @title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [24]:
# @title Install dependencies

from UNet import *

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# @title Plotting Utils

def plot_state(state, prediction=None, target=None):
    add = 0
    if prediction is not None:
        add += 1
    if target is not None:
        add += 1
        
    fig, ax = plt.subplots(1, state.shape[0] + add, figsize=(20, 5))
    for i in range(state.shape[0]):
        ax[i].imshow(state[i])
        ax[i].set_title(f'Input {i}')
        plt.colorbar(ax[i].imshow(state[i]), ax=ax[i])

    if prediction is not None:
        ax[state.shape[0]].imshow(prediction)
        ax[state.shape[0]].set_title('Prediction')
        plt.colorbar(ax[state.shape[0]].imshow(prediction), ax=ax[state.shape[0]])
                     
    if target is not None:
        ax[state.shape[0]- 1 + add].imshow(target)
        ax[state.shape[0]- 1 + add].set_title('Target')
        plt.colorbar(ax[state.shape[0] - 1 + add].imshow(target), ax=ax[state.shape[0]- 1 + add])

    plt.show()

def rollout(model, current_state, steps, input_shape, plot_shape, targets, plot = False):
    output = []
    for i in range(steps):
        pred = model.predict(current_state.reshape(input_shape)).reshape(plot_shape)
        if plot:
            plot_state(current_state[0], prediction=pred, target=targets[i])
        current_state = current_state[:,1:]
        current_state = np.append(current_state, pred.reshape(1, 1, pred.shape[0], pred.shape[1]), axis=1)
        output.append(pred)

    return np.array(output).squeeze()

In [3]:
# @title RRNet Utils

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate, add


def conv_block(input, num_filters, kernel_size=3):
    x = Conv2D(num_filters, kernel_size, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, kernel_size, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters, kernel_size_=3):
    x = conv_block(input, num_filters, kernel_size_)
    p = MaxPool2D((2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters, kernel_size_=3):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters, kernel_size_)
    return x

def RR_block(input, num_filters, kernel_size=3, stack_num=2, recur_num=2):
    x0 = Conv2D(num_filters, kernel_size, padding="same")(input)
    x = x0
    
    for i in range(stack_num):

        x_res = Conv2D(num_filters, kernel_size, padding="same")(x)
        x_res = BatchNormalization()(x_res)
        x_res = Activation("relu")(x_res)
            
        for j in range(recur_num):
            x_add = add([x_res, x])

            x_res = Conv2D(num_filters, kernel_size, padding="same")(x_add)
            x_res = BatchNormalization()(x_res)
            x_res = Activation("relu")(x_res)
            
        x = x_res

    x_out = add([x, x0])

    return x_out

def RR_encoder_block(input, num_filters, kernel_size_=3):
    x = RR_block(input, num_filters, kernel_size_)
    p = MaxPool2D((2, 2))(x)
    return x, p 

def RR_decoder_block(input, skip_features, num_filters, kernel_size_=3):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = RR_block(x, num_filters, kernel_size_)
    return x


#Build Unet using the blocks
def build_unet(input_shape, steps = 1):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(steps, 1, padding="same", activation="sigmoid")(d4)  #Binary (can be multiclass)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [6]:
# @title Load Data

ds = xr.open_dataset('/content/drive/MyDrive/ERA5_Data/2023_SouthAfrica.nc')
ds.load()

ds['ws'] = (ds.u10**2 + ds.v10**2)**0.5

In [7]:
# @title Data Preperation

window_size = 2
variable = 'ws'

X, y = [], []

for i in range(ds.time.size - window_size):
    X.append(ds[variable].isel(time=slice(i, i + window_size)))
    y.append(ds[variable].isel(time=i + window_size))

X_ = np.array(X)
y_ = np.array(y)

# Train Validation Split
split = round(X_.shape[0] * 0.9)

# Shape fixing
# lon_shape = ds.longitude.size
# lat_shape = ds.latitude.size

# Custom Shapes for the data
lon_shape = 64
lat_shape = 32
X_ = X_[:,:, 1:33, 3:67]
y_ = y_[:,1:33,3:67]

# Normalization
max_ws = X_.max()
X_n = X_ / max_ws
y_n = y_ / max_ws

# Splitting
f_train, t_train = X_n[:split].astype('float32'), y_n[:split].astype('float32')
f_test, t_test = X_n[split:].astype('float32'), y_n[split:].astype('float32')
print('Train Shape: ', f_train.shape, t_train.shape, '\n\nTest Shape: ', f_test.shape, t_test.shape)

X_n = X_n.reshape(-1, lat_shape, lon_shape, window_size)
y_n = y_n.reshape(-1, lat_shape, lon_shape,  1)

f_training, t_training = X_n[:split].astype('float32'), y_n[:split].astype('float32')
f_testing, t_testing = X_n[split:].astype('float32'), y_n[split:].astype('float32')

Train Shape:  (625, 3, 16, 48) (625, 16, 48) 

Test Shape:  (69, 3, 16, 48) (69, 16, 48)


In [26]:
# @title Build RRNet

def build_RRnet(input_shape, steps = 1):
    inputs = Input(input_shape)

    s1, p1 = RR_encoder_block(inputs, 16)
    s2, p2 = RR_encoder_block(p1, 32)
    # s3, p3 = encoder_block(p2, 256)
    # s4, p4 = encoder_block(p3, 512)

    b1 = RR_block(p2, 64) #Bridge

    # d1 = decoder_block(b1, s4, 512)
    # d2 = decoder_block(d1, s3, 256)
    d3 = RR_decoder_block(b1, s2, 32)
    d4 = RR_decoder_block(d3, s1, 16)

    outputs = Conv2D(steps, 1, padding="same", activation="sigmoid")(d4)  #Binary (can be multiclass)

    model = Model(inputs, outputs, name="U-Net")
    return model

input_shape = tuple((1, lat_shape, lon_shape, window_size))
plot_shape = t_training[0].shape

model = build_RRnet(input_shape[1:])

model.summary()

In [28]:
# @title Train RRNet

batch_size = 64
epochs = 500
model.compile(optimizer='adam', loss='mse')

early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/ERA5_Data/2023_SouthAfrica.nc/RRNet.h5', monitor='val_loss', save_best_only=True, verbose=1, mode='min')

if True:
    history = model.fit(
        f_training,
        t_training,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(f_testing, t_testing),
        callbacks=[early_stopping, model_checkpoint]
    )

Epoch 1/500
5/5 [==============================] - ETA: 0s - loss: 0.1001
Epoch 1: val_loss improved from inf to 0.08565, saving model to synth_RR
INFO:tensorflow:Assets written to: synth_RR\assets


INFO:tensorflow:Assets written to: synth_RR\assets


5/5 [==============================] - 26s 4s/step - loss: 0.1001 - val_loss: 0.0856
Epoch 2/500
5/5 [==============================] - ETA: 0s - loss: 0.0410
Epoch 2: val_loss did not improve from 0.08565
5/5 [==============================] - 2s 399ms/step - loss: 0.0410 - val_loss: 0.1060
Epoch 3/500
5/5 [==============================] - ETA: 0s - loss: 0.0246
Epoch 3: val_loss did not improve from 0.08565
5/5 [==============================] - 2s 380ms/step - loss: 0.0246 - val_loss: 0.1434
Epoch 4/500
5/5 [==============================] - ETA: 0s - loss: 0.0167
Epoch 4: val_loss did not improve from 0.08565
5/5 [==============================] - 2s 429ms/step - loss: 0.0167 - val_loss: 0.1455
Epoch 5/500
5/5 [==============================] - ETA: 0s - loss: 0.0128
Epoch 5: val_loss did not improve from 0.08565
5/5 [==============================] - 2s 479ms/step - loss: 0.0128 - val_loss: 0.1465
Epoch 6/500
5/5 [==============================] - ETA: 0s - loss: 0.0104
Epoch 6: 

INFO:tensorflow:Assets written to: synth_RR\assets


5/5 [==============================] - 13s 3s/step - loss: 0.0046 - val_loss: 0.0744
Epoch 12/500
5/5 [==============================] - ETA: 0s - loss: 0.0041
Epoch 12: val_loss improved from 0.07442 to 0.06442, saving model to synth_RR
INFO:tensorflow:Assets written to: synth_RR\assets


...

INFO:tensorflow:Assets written to: synth_RR\assets


5/5 [==============================] - 11s 3s/step - loss: 0.0027 - val_loss: 0.0250
Epoch 28/500
5/5 [==============================] - ETA: 0s - loss: 0.0027
Epoch 28: val_loss improved from 0.02498 to 0.02441, saving model to synth_RR
INFO:tensorflow:Assets written to: synth_RR\assets


INFO:tensorflow:Assets written to: synth_RR\assets


5/5 [==============================] - 13s 3s/step - loss: 0.0027 - val_loss: 0.0244
Epoch 29/500
5/5 [==============================] - ETA: 0s - loss: 0.0025
Epoch 29: val_loss improved from 0.02441 to 0.02360, saving model to synth_RR


KeyboardInterrupt: 

In [2]:
# @title Test RRNet

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# model.load_weights('models/model.h5')
model.load_weights('/content/drive/MyDrive/ERA5_Data/2023_SouthAfrica.nc/RRNet.h5')
seed = 60
steps = 24

targets = t_test[seed:steps + seed]

rollout_pred = rollout(model, f_test[seed:seed+ 1], steps, input_shape, plot_shape, targets, plot=False)

rollout_pred = rollout_pred * max_ws
targets = targets * max_ws

mse = mean_squared_error(targets.flatten(), rollout_pred.flatten())
mae = mean_absolute_error(targets.flatten(), rollout_pred.flatten())
mape = mean_absolute_percentage_error(targets.flatten(), rollout_pred.flatten())
rmse = np.sqrt(mse)

print(f'Mean Squared Error: {mse}, Mean Absolute Error: {mae}, Mean Absolute Percentage Error: {mape}, Root Mean Squared Error: {rmse}')

NameError: name 'model' is not defined